In [1]:
import azure.core
from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [2]:
resource_name = "mds_rg"
workspace_name = "mds_ws01"
subscriptionID = "xxxxxxx" 
aml_compute_target = "task3p-compute" 
experiment_name= "task3_expirement"

In [52]:
# create Workspace
try:
    ws=Workspace.from_config()
    print('Workspace already exists')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_name,
                    create_resource_group=True,
                    subscription_id=subscriptionID,
                    location="East US")
    ws.write_config('.azureml')


Workspace already exists


In [4]:
Workspace.from_config()

Workspace.create(name='mds_ws01', subscription_id='xxxxxxxxxx', resource_group='mds_rg')

In [53]:
# Create Compute Target
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Compute Target {} already exist.".format(aml_compute_target))
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_DS1_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=3000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached...")

Compute Target task3p-compute already exist.
Azure Machine Learning Compute attached...


In [6]:
# Create Experiment
exp = Experiment(ws,experiment_name)

In [ ]:
# Create environment to execute your code
env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml")

In [8]:
conda install scikit-learn

Solving environment: - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 22.11.1
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.3.1



## Package Plan ##

  environment location: /anaconda/envs/jupyter_env

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    appdirs-1.4.4              |     pyhd3eb1b0_0          12 KB
    blas-1.0                   |              mkl           6 KB
    brotlipy-0.7.0             |py38h27cfd23_1003         323 KB
    ca-certificates-2023.01.10 |       h06a4308_0         120 KB
    certifi-2022.12.7          |   py38h06a4308_0         150 KB
    cffi-1.15.1                |   py38h74dc2b5_0 

In [12]:
config=ScriptRunConfig(source_directory="./",script="model_train.py",compute_target=aml_compute_target,
                            environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7])
execution=exp.submit(config)

In [16]:
#Register model in workspace
from azureml.core import Model
model = Model.register(ws,model_path="./diabetes_model_rf.pkl",model_name="task3p_rfmodel")

Registering model task3p_rfmodel


In [17]:
env.environment_variables

{'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE'}

In [18]:
model = Model(ws,"task3p_rfmodel")

In [88]:
model.id

'task3p_model:1'

In [30]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="azure_mlenv")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="./score_model.py",
)

In [54]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [55]:
service = Model.deploy(
    ws,
    "task3pservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_7295/75091019.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry e4dd1454ef2348cd8553569697732ace.azurecr.io
Logging into Docker registry e4dd1454ef2348cd8553569697732ace.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM e4dd1454ef2348cd8553569697732ace.azurecr.io/azureml/azureml_1c2aabfbb77e8e728d98c56c085c208f
 ---> bb1dd9ac823f
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> d25c42bdd3f8
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImY5YWYzMTk5LTBkZDgtNDI4ZS04YTUwLWMxM2QwZDE3MWE2NiIsInJlc291cmNlR3JvdXBOYW1lIjoibWRzX3JnIiwiYWNjb3VudE5hbWUiOiJtZHNfd3MwMSIsIndvcmtzcGFjZUlkIjoiZTRkZDE0NTQtZWYyMy00OGNkLTg1NTMtNTY5Njk3NzMyYWNlIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 869677807b52
 ---> 2e5efd7052bb
Step 4/5 : RUN mv '/var/azureml-app/tmp_q827i2_.py' /var/azureml-app/main.py
 ---> Running in ca19c93fd928
 ---> 31ab

In [33]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-04-02T03:06:30,978720118+00:00 - iot-server/run 
2023-04-02T03:06:30,982607859+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/lib/libtinfo.so.6: no version information available (required by bash)
2023-04-02T03:06:31,014848397+00:00 - nginx/run 
2023-04-02T03:06:31,027924335+00:00 - gunicorn/run 
2023-04-02T03:06:31,049844465+00:00 | gunicorn/run | 
2023-04-02T03:06:31,076477545+00:00 | gu

In [56]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    "task_name": "task 3",
    "grade": "distinction"
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

test is {'task_name': 'task 3', 'grade': 'distinction'}


In [35]:
env = Environment(name='azure_mlenv')
python_packages = ['azureml', 'numpy', 'scikit-learn']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inference_config = InferenceConfig(environment=env, source_directory='./source_dir', 
                                    entry_script='./score_diabmodel.py')

In [39]:
service = Model.deploy(
    ws,
    "task3pservice",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_7295/1845266671.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry e4dd1454ef2348cd8553569697732ace.azurecr.io
Logging into Docker registry e4dd1454ef2348cd8553569697732ace.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM e4dd1454ef2348cd8553569697732ace.azurecr.io/azureml/azureml_aae5dc9135e6cccda9295b605bf6aacf
 ---> 266d4041d3b0
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 2eb753a9b04d
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImY5YWYzMTk5LTBkZDgtNDI4ZS04YTUwLWMxM2QwZDE3MWE2NiIsInJlc291cmNlR3JvdXBOYW1lIjoibWRzX3JnIiwiYWNjb3VudE5hbWUiOiJtZHNfd3MwMSIsIndvcmtzcGFjZUlkIjoiZTRkZDE0NTQtZWYyMy00OGNkLTg1NTMtNTY5Njk3NzMyYWNlIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 9304d259d043
 ---> 42a9907498df
Step 4/5 : RUN mv '/var/azureml-app/tmpkjza6qtg.py' /var/azureml-app/main.py
 ---> Running in 687d270a5ad4
 ---> aed4

In [40]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_7df3962287fe0686a7fc2db5566e0582/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_7df3962287fe0686a7fc2db5566e0582/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_7df3962287fe0686a7fc2db5566e0582/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_7df3962287fe0686a7fc2db5566e0582/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-04-02T06:47:29,850638316+00:00 - rsyslog/run 
2023-04-02T06:47:29,850318813+00:00 - iot-server/run 
bash: /azureml-envs/azureml_7df3962287fe0686a7fc2db5566e0582/lib/libtinfo.so.6: no version information available (required by bash)
2023-04-02T06:47:29,866320080+00:00 - nginx/run 
2023-04-02T06:47:29,886640693+00:00 - gunicorn/run 
2023-04-02T06:47:29,899111623+00:00 | gunicorn/run | 
2023-04-02T06:47:29,914195981+00:00 | gu

In [44]:
# Prepare your test data
import pandas as pd
import json
data = pd.read_csv('test_sample.csv')
xt = data.values.tolist()
test = json.dumps({"data":xt})

In [46]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}

response = requests.post(uri, data=test, headers=headers)
print("Predicted value :", response.json())

Predicted value : [1]


In [47]:
data = pd.read_csv('test.csv')
xt = data.values.tolist()
test = json.dumps({"data":xt})

In [48]:
response = requests.post(uri, data=test, headers=headers)
print("Predicted values are :")
print(response.json())

Predicted values are :
[1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0

In [49]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [50]:
service = Model.deploy(
    ws,
    "task3pservice",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_7295/1845266671.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-04-02 07:18:59+00:00 Creating Container Registry if not exists.
2023-04-02 07:18:59+00:00 Registering the environment.
2023-04-02 07:19:01+00:00 Use the existing image.
2023-04-02 07:19:01+00:00 Generating deployment configuration.
2023-04-02 07:19:02+00:00 Submitting deployment to compute.
2023-04-02 07:19:07+00:00 Checking the status of deployment task3pservice..
2023-04-02 07:20:30+00:00 Checking the status of inference endpoint task3pservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
print(service.get_logs())

In [51]:
from azureml.core import Webservice

service = Webservice(workspace=ws, name="task3pservice")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response and logs

data = pd.read_csv('test_sample.csv')
xt = data.values.tolist()
test = json.dumps({"data":xt})

resp = requests.post(scoring_uri, data=test, headers=headers)

print("Predicted value :", resp.json())

Predicted value : [1]
